<a href="https://colab.research.google.com/github/acergamebook/3DMark-for-D3D11-in-VM/blob/master/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [102]:
from urllib.request import urlretrieve
url= "https://github.com/Elwing-Chou/acerml/raw/main/titanic/train.csv"
urlretrieve(url, "train.csv")

url= "https://github.com/Elwing-Chou/acerml/raw/main/titanic/test.csv"
urlretrieve(url, "test.csv")

('test.csv', <http.client.HTTPMessage at 0x7ff5b6e1a6a0>)

In [103]:
import pandas as pd
train_df=pd.read_csv("train.csv",encoding="utf-8")
test_df=pd.read_csv("test.csv",encoding="utf-8")

In [104]:
test_df

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [105]:
#axis=0, row. axis=1, columns
datas = pd.concat([train_df,test_df],axis=0,ignore_index=True)
datas = datas.drop(["PassengerId","Survived"], axis=1)

In [106]:
#Series[True/False list]
s = datas.isna().sum()
s[s > 0].sort_values(ascending=False)

Cabin       1014
Age          263
Embarked       2
Fare           1
dtype: int64

In [107]:
#類別裡面最常出現的值
most=datas["Embarked"].value_counts().idxmax()
datas["Embarked"]=datas["Embarked"].fillna(most)

In [108]:
#數值補中位數
med=datas.median().drop(["Pclass"])
datas=datas.fillna(med)
s=datas.isna().sum()
s[s>0].sort_values(ascending=False)

Cabin    1014
dtype: int64

In [109]:
def cabinflow(c):
  if pd.isna(c):
    return c
  else:
    return c[0]
datas["Cabin"] = datas["Cabin"].apply(cabinflow)
datas["Cabin"].value_counts()

C    94
B    65
D    46
E    41
A    22
F    21
G     5
T     1
Name: Cabin, dtype: int64

In [110]:
dic=datas["Ticket"].value_counts()
def ticketflow(t):
    return dic[t]
datas["Ticket"]=datas["Ticket"].apply(ticketflow)
datas["Ticket"].value_counts()

1     713
2     264
3     147
4      64
7      35
5      35
6      24
8      16
11     11
Name: Ticket, dtype: int64

In [111]:
def nameflow(n):
    mid = n.split(".")[0].split(",")[-1]
    return mid.strip()

middle = datas["Name"].apply(nameflow).value_counts()
whitelist = middle.index[:4]

def nameflow2(n):
    mid = nameflow(n)
    if mid in whitelist:
        return mid
    else:
        return None
datas["Name"] = datas["Name"].apply(nameflow2)
datas["Name"].value_counts()

Mr        757
Miss      260
Mrs       197
Master     61
Name: Name, dtype: int64

In [112]:
#One-Hot Encoding
datas = pd.get_dummies(datas)
datas = pd.get_dummies(datas, columns=["Pclass"])


In [113]:
datas["Family"] = datas["SibSp"] + datas["Parch"]

In [117]:
#列：iloc (列編號), loc(列標簽)
#df.iloc -> [1st, 2nd...]

x_train = datas.iloc[:len(train_df)]
#print(x_train)
x_test = datas.iloc[len(train_df):]
y_train = train_df["Survived"]


from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
clf = RandomForestClassifier(n_estimators=20, max_depth=5)
params = {
    "max_depth":[7], "n_estimators":range(20,100)

}
cv=GridSearchCV(clf, params, cv=10, n_jobs=-1)
cv.fit(x_train,y_train)
print(cv.best_score_)
print(cv.best_params_)
import numpy as np
from sklearn.model_selection import cross_val_score

scores=cross_val_score(clf,x_train, y_train, cv=10,n_jobs=-1)
print("10 times: ", scores)
print("average: ", np.average(scores))

0.8440199750312111
{'max_depth': 7, 'n_estimators': 99}
10 times:  [0.84444444 0.85393258 0.76404494 0.87640449 0.84269663 0.83146067
 0.83146067 0.7752809  0.85393258 0.85393258]
average:  0.8327590511860175
